In [135]:
!pip install openpyxl 
!pip install -U nltk
!pip install pymysql

import numpy as np
import pandas as pd
# sudo apt-get install python3-pymysql
from pandas import read_excel
import seaborn as sns
import statsmodels.api as sm
import sqlalchemy
import pymysql
import matplotlib.pyplot as plt

from sqlalchemy import create_engine

from statistics import *
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.decomposition import PCA
from matplotlib import style
from mpl_toolkits.mplot3d import Axes3D



Requirement already up-to-date: nltk in /usr/local/lib/python3.6/dist-packages (3.5)


In [136]:
from google.colab import drive
drive.mount ('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [137]:
my_sheet = 'Sheet1' # change it to your sheet name
file_name = 'drive/My Drive/Week2_challenge_data_source.xlsx' # change it to the name of your excel file
df = read_excel(file_name, sheet_name = my_sheet, usecols="H,AD,AE")
# df

##Handsets


In [138]:
df['Handset Type'].value_counts()[df['Handset Type'].value_counts() == df['Handset Type'].value_counts().max()]

Huawei B528S-23A    19752
Name: Handset Type, dtype: int64

In [139]:
#top 10 handset types
n = 10
df['Handset Type'].value_counts()[:n].index.tolist()

['Huawei B528S-23A',
 'Apple iPhone 6S (A1688)',
 'Apple iPhone 6 (A1586)',
 'undefined',
 'Apple iPhone 7 (A1778)',
 'Apple iPhone Se (A1723)',
 'Apple iPhone 8 (A1905)',
 'Apple iPhone Xr (A2105)',
 'Samsung Galaxy S8 (Sm-G950F)',
 'Apple iPhone X (A1901)']

In [140]:
#top 3 handset manufacturers
n = 3
top_manufacturer_df=df['Handset Manufacturer'].value_counts()[:n].index.tolist()
top_manufacturer_df

['Apple', 'Samsung', 'Huawei']

In [141]:
df.groupby(['Handset Manufacturer', 'Handset Type']).size().reset_index(name='count').sort_values(['Handset Type','Handset Manufacturer']).groupby('Handset Manufacturer').head(3)

,Handset Manufacturer,Handset Type,count
0,A-Link Telecom International Co Limited,A-Link Telecom I. Cubot A5,1
1,A-Link Telecom International Co Limited,A-Link Telecom I. Cubot Note Plus,1
2,A-Link Telecom International Co Limited,A-Link Telecom I. Cubot Note S,1
14,Acer,Acer Liquid M220,1
15,Acer,Acer Liquid Z220,1
...,...,...,...
1372,ZTE,Zte Blade A452 Blade A452,1
1393,Zyxel Communications Corp,Zyxel Communicat. Lte7460,1
1394,Zyxel Communications Corp,Zyxel Communicat. Sbg3600,1
1392,ZyXEL Communications Corp,Zyxel Communicat. Zyxel Wah7706,1


In [142]:
df1=df.groupby(["Handset Manufacturer", "Handset Type"]).size().reset_index(name="count").sort_values(["Handset Manufacturer","count"],ascending=[1,0]).groupby('Handset Manufacturer').head(5)
df1.to_csv('out3.csv')
df1

,Handset Manufacturer,Handset Type,count
6,A-Link Telecom International Co Limited,A-Link Telecom I. Cubot X18 Plus,2
0,A-Link Telecom International Co Limited,A-Link Telecom I. Cubot A5,1
1,A-Link Telecom International Co Limited,A-Link Telecom I. Cubot Note Plus,1
2,A-Link Telecom International Co Limited,A-Link Telecom I. Cubot Note S,1
3,A-Link Telecom International Co Limited,A-Link Telecom I. Cubot Nova,1
...,...,...,...
1377,Zte,Zte Axon 6S Zte Axon 7 Axon 7,1
1392,ZyXEL Communications Corp,Zyxel Communicat. Zyxel Wah7706,1
1393,Zyxel Communications Corp,Zyxel Communicat. Lte7460,1
1394,Zyxel Communications Corp,Zyxel Communicat. Sbg3600,1


In [143]:
my_sheet = 'Sheet1' 
file_name = 'drive/My Drive/Week2_challenge_data_source.xlsx' 
df_user = read_excel(file_name, sheet_name = my_sheet, usecols="A,H,F,K,L,M,N,O,P,AE,BB,BC,AN,AO,AP,AQ,AR,AS,AT,AU,AV,AW,AX,AY,AZ,BA,BB,BC")
# df_user

In [144]:
Q1 = df_user.quantile(0.25)
Q3 = df_user.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

Dur. (ms)                      7.498975e+04
MSISDN/Number                  3.219519e+07
Avg RTT DL (ms)                3.800000e+01
Avg RTT UL (ms)                1.300000e+01
Avg Bearer TP DL (kbps)        1.966775e+04
Avg Bearer TP UL (kbps)        1.073000e+03
TCP DL Retrans. Vol (Bytes)    3.732657e+06
TCP UL Retrans. Vol (Bytes)    7.932550e+04
Social Media DL (Bytes)        1.795790e+06
Social Media UL (Bytes)        3.288600e+04
Google DL (Bytes)              5.741159e+06
Google UL (Bytes)              2.064175e+06
Email DL (Bytes)               1.796534e+06
Email UL (Bytes)               4.670570e+05
Youtube DL (Bytes)             1.161502e+07
Youtube UL (Bytes)             1.099760e+07
Netflix DL (Bytes)             1.169332e+07
Netflix UL (Bytes)             1.103129e+07
Gaming DL (Bytes)              4.227009e+08
Gaming UL (Bytes)              8.303148e+06
Other DL (Bytes)               4.215049e+08
Other UL (Bytes)               8.238205e+06
Total UL (Bytes)               1

In [145]:
# print(df_user < (Q1 - 1.5 * IQR)) |(df_user > (Q3 + 1.5 * IQR))


##SOCIAL MEDIA

In [146]:
#total volume of social media in bytes
sum_column = df_user["Social Media UL (Bytes)"] + df_user["Social Media DL (Bytes)"]
sum_data=df_user['Total DL (Bytes)']+df_user['Total UL (Bytes)']
df_user["Total Social Media Volume (Bytes)"] = sum_column
df_user['Total UL+Dl (Bytes)']=sum_data
df_user.to_csv("social_media.csv")
sm_df=df_user.groupby('MSISDN/Number')['Total Social Media Volume (Bytes)'].sum().reset_index(name='Total_Social_Media_Volume')
# sm_df

# #min,max, average of social media 
# sm_df['Total_Social_Media_Volume'].describe()

#plot
# (sm_df['Total_Social_Media_Volume'].value_counts().head(10) / len(sm_df)).plot.bar()
# sns.distplot(sm_df.Total_Social_Media_Volume.dropna(), kde=False, bins = 50)

#top 10 users
sm_df_1=sm_df.nlargest(10,'Total_Social_Media_Volume')
sm_df_1.to_csv("socialmedia10.csv")





##GOOGLE

In [147]:
#total volume of google in bytes
sum_column1 = df_user["Google UL (Bytes)"] + df_user["Google DL (Bytes)"]
sum_data1=df_user['Total DL (Bytes)']+df_user['Total UL (Bytes)']
df_user["Total Google Volume (Bytes)"] = sum_column1
df_user['Total UL+Dl (Bytes)']=sum_data1
df_user.to_csv("google.csv")
gg_df=df_user.groupby('MSISDN/Number')['Total Google Volume (Bytes)'].sum().reset_index(name='Total_Google_Volume')
# gg_df
# #min,max, average of google 
gg_df['Total_Google_Volume'].describe()

#plot
# (gg_df['Total Google Volume (Bytes)'].value_counts().head(10) / len(sm_df)).plot.bar()
# sns.distplot(gg_df.Total_Google_Volume.dropna(), kde=False, bins = 50)


#top 10 users
gg_df_1=gg_df.nlargest(10,'Total_Google_Volume')
gg_df_1.to_csv("google10.csv")




##EMAIL

In [148]:
#total volume of email in bytes
sum_column2 = df_user["Email UL (Bytes)"] + df_user["Email DL (Bytes)"]
sum_data2=df_user['Total DL (Bytes)']+df_user['Total UL (Bytes)']
df_user['Total UL+Dl (Bytes)']=sum_data2
df_user["Total Email Volume (Bytes)"] = sum_column2
df_user.to_csv("email.csv")
em_df=df_user.groupby('MSISDN/Number')['Total Email Volume (Bytes)'].sum().reset_index(name='Total Email volume (Bytes)')
# em_df

# #min,max, average of email 
# em_df['Total Email volume (Bytes)'].describe()


#plot
# (em_df['Total Email volume (Bytes)'].value_counts().head(10) / len(sm_df)).plot.bar()


#top 10 users
em_df_1=em_df.nlargest(10,'Total Email volume (Bytes)')
em_df_1.to_csv("email10.csv")




##Netflix

In [149]:
#total volume of netflix in bytes
sum_colum = df_user["Netflix UL (Bytes)"] + df_user["Netflix DL (Bytes)"]
sum_dat=df_user['Total DL (Bytes)']+df_user['Total UL (Bytes)']
df_user['Total UL+Dl (Bytes)']=sum_dat
df_user["Total Netflix Volume (Bytes)"] = sum_colum
df_user.to_csv("email.csv")
nf_df=df_user.groupby('MSISDN/Number')['Total Netflix Volume (Bytes)'].sum().reset_index(name='Total Netflix volume (Bytes)')
# nf_df

# #min,max, average of netflix 
# nf_df['Total Netflix volume (Bytes)'].describe()


#plot
# (nf_df['Total Netflix volume (Bytes)'].value_counts().head(10) / len(sm_df)).plot.bar()


#top 10 users
nf_df_1=nf_df.nlargest(10,'Total Netflix volume (Bytes)')
nf_df_1.to_csv("netflix10.csv")




##YOUTUBE

In [150]:
#total volume of youtube in bytes
sum_column3 = df_user["Youtube UL (Bytes)"] + df_user["Youtube DL (Bytes)"]
sum_data3=df_user['Total DL (Bytes)']+df_user['Total UL (Bytes)']
df_user["Total Youtube Volume (Bytes)"] = sum_column3
df_user['Total UL+Dl (Bytes)']=sum_data3
df_user.to_csv("youtube.csv")
yt_df=df_user.groupby('MSISDN/Number')['Total Youtube Volume (Bytes)'].sum().reset_index(name='Total Youtube Volume (Bytes)')
# yt_df

# #min,max, average of Youtube 
# yt_df['Total Youtube Volume (Bytes)'].describe()

#plot
# (yt_df['Total Youtube Volume (Bytes)'].value_counts().head(10) / len(sm_df)).plot.bar()

#top 10 users
yt_df_1=yt_df.nlargest(10,'Total Youtube Volume (Bytes)')
yt_df_1.to_csv("youtube10.csv")



##GAMING APPLICATION

In [151]:
#total volume of gaming in bytes
sum_column5 = df_user["Gaming UL (Bytes)"] + df_user["Gaming DL (Bytes)"]
sum_data5=df_user['Total DL (Bytes)']+df_user['Total UL (Bytes)']
df_user['Total UL+Dl (Bytes)']=sum_data5
df_user["Total Gaming Volume (Bytes)"] = sum_column5
df_user.to_csv("gaming.csv")
gm_df=df_user.groupby('MSISDN/Number')['Total Gaming Volume (Bytes)'].sum().reset_index(name='Total Gaming Volume (Bytes)')
# gm_df

# #min,max, average of Gaming 
gm_df['Total Gaming Volume (Bytes)'].describe()

#plot
# (gm_df['Total Gaming Volume (Bytes)'].value_counts().head(10) / len(sm_df)).plot.bar()

gm_df_1=gm_df.nlargest(10,'Total Gaming Volume (Bytes)')
gm_df_1.to_csv("gaming10.csv")


##OTHER

In [152]:
#total volume of other in bytes
sum_column = df_user["Other UL (Bytes)"] + df_user["Other DL (Bytes)"]
df_user["Total Other Volume (Bytes)"] = sum_column
other_df=df_user.groupby('MSISDN/Number')['Total Other Volume (Bytes)'].sum().reset_index(name="Total Other Volume(Bytes)")
other_df


,MSISDN/Number,Total Other Volume(Bytes)
0,3.360100e+10,386570872.0
1,3.360100e+10,281710071.0
2,3.360100e+10,501693672.0
3,3.360101e+10,35279702.0
4,3.360101e+10,804804484.0
...,...,...
106851,3.379000e+10,295282817.0
106852,3.379000e+10,211115113.0
106853,3.197021e+12,470526473.0
106854,3.370000e+14,149188439.0


In [153]:
top_3_apps=[gm_df["Total Gaming Volume (Bytes)"],sm_df()]

TypeError: ignored

##XDR SESSIONS

In [ ]:
#number of xdr sessions per user
xdr=df_user.groupby('MSISDN/Number')['Bearer Id'].count().reset_index(name='count')
# xdr.to_csv("xdr.csv")
xdr

#min,max, average of xdr sessions
xdr['count'].describe()

# #PLOT
# # (xdr['count'].value_counts().head(10) / len(sm_df)).plot.bar()

# #top 10 customers per session duration metric
# # 
# xdr1=xdr.nlargest(10,'count')
# xdr1.to_csv("sessions_10.csv")


##SESSION DURATION

In [ ]:
#session duration per user
dur_agg=df_user.groupby('MSISDN/Number')['Dur. (ms)'].sum().reset_index(name="sum")
dur_agg

#min,max, average of sessions duration
dur_agg["sum"].describe()

#plot
(dur_agg['sum'].value_counts().head(10) / len(sm_df)).plot.bar()

#top 10 customers per session duration metric
dur_agg1=dur_agg.nlargest(10,'sum')
dur_agg1.to_csv("duration_10.csv")

##UL+DL

In [ ]:
#total UL data per user
dur_agg_ul=df_user.groupby('MSISDN/Number')['Total UL (Bytes)'].sum().reset_index(name="sum")
dur_agg_ul

# #min,max, average of UL sessions
dur_agg_ul['sum'].describe()
#plot
# (dur_agg_ul['sum'].value_counts().head(10) / len(sm_df)).plot.bar()

In [ ]:
#top 10 customers per total Ul and DL metric
sum_data7=df_user['Total DL (Bytes)']+df_user['Total UL (Bytes)']
df_user['Total UL+DL']=sum_data7
agg_ul_dl=df_user.groupby('MSISDN/Number')['Total UL+DL'].sum().reset_index(name="sum")
# agg_ul_dl
agg_ul_dl1=agg_ul_dl.nlargest(10,'sum')
agg_ul_dl1.to_csv("total_data10.csv")



# agg_ul_dl.plot(x="MSISDN/Number", y=["sum"], kind="bar")
# plt.show
sns.barplot(x="MSISDN/Number",y="sum",data=agg_ul_dl1)
plt.show()


In [ ]:
#total DL data per user
dur_agg_dl=df_user.groupby('MSISDN/Number')['Total DL (Bytes)'].sum().reset_index(name="sum")
# dur_agg_dl

# #min,max, average of DL sessions
dur_agg_dl['sum'].describe()

#@title Default title text
(dur_agg_dl['sum'].value_counts().head(10) / len(sm_df)).plot.bar()



##K means engagement metrics

In [ ]:
#engagament metrics kmeans clustering
data_eng = [xdr["count"], dur_agg["sum"],agg_ul_dl["sum"]]
headers = ["session frequency", "Duration",
          "Total Data" ]     
eng_metrics = pd.concat(data_eng, axis=1, keys=headers)
eng_metrics

kmeans = KMeans(n_clusters=3)

y = kmeans.fit_predict(eng_metrics[['session frequency', 'Duration', 'Total Data',]])

eng_metrics['Cluster'] = y

eng000 = eng_metrics[eng_metrics.Cluster==0]
eng000
eng111 =  eng_metrics[eng_metrics.Cluster==1]
eng111
eng222 =  eng_metrics[eng_metrics.Cluster==2]
eng222




In [ ]:
#normalize
norm_metrics=(eng_metrics-eng_metrics.min())/(eng_metrics.max()-eng_metrics.min())

kmeans = KMeans(n_clusters=3)

y = kmeans.fit_predict(norm_metrics[['session frequency', 'Duration', 'Total Data',]])

norm_metrics['Cluster'] = y
norm_metrics

eng00 = norm_metrics[norm_metrics.Cluster==0]
eng00
eng11 =  norm_metrics[norm_metrics.Cluster==1]
eng11
eng22 =  norm_metrics[norm_metrics.Cluster==2]
eng22


In [ ]:
e1=eng000.describe()
e1.to_csv("engoo.csv")
e1


In [ ]:
e2=eng111.describe()
e2.to_csv("eng11.csv")


In [ ]:
e3=eng222.describe()
e3.to_csv("eng22.csv")


In [ ]:
alldistances = kmeans.fit_transform(norm_metrics)
# alldistances
ddf=pd.DataFrame(alldistances)
ddf.columns = ['0','1','2']
ddf

In [ ]:
norm_metrics['Cluster'].value_counts()

##engagement score

In [ ]:
#engagemnt score

engagement_data= [df_user["MSISDN/Number"],ddf["1"]]
engagement_data
# headers = ["MSISDN/Number","engagement_score"]  
# engagement_score=pd.concat(engagement_data,axis=1,keys=headers)
# engagement_score


In [ ]:
data_eng1 = [xdr["MSISDN/Number"],xdr["count"], dur_agg["sum"],agg_ul_dl["sum"],norm_metrics["Cluster"]]
headers1 = ["Customer Number","session frequency", "Duration",
          "Total Data","Cluster" ] 
non_norm=pd.concat(data_eng1,axis=1,keys=headers1)
non_norm

In [ ]:
nn00 = non_norm[non_norm.Cluster==0]
nn00.describe()


In [ ]:
nn11 =  non_norm[non_norm.Cluster==1]
nn11.describe()


In [ ]:
nn22 =  non_norm[non_norm.Cluster==2]
nn22.describe()

In [ ]:
sns.lmplot(x='session frequency',y='Duration',data=norm_metrics,hue='Cluster',fit_reg=False)


##DECILE

In [ ]:
#decile
dur_agg['Decile_rank']=pd.qcut(dur_agg['sum'],10,labels=False)
print(dur_agg)


In [ ]:
decile1 = dur_agg[dur_agg['Decile_rank']==0]
sum_data22=df_user['Total DL (Bytes)']+df_user['Total UL (Bytes)']
df_user['Total UL+DL']=sum_data22
numbers1 = df_user["Total UL+DL"]
decile1 = decile1.join(numbers1)
decile1.loc['Total'] = pd.Series(decile1['Total UL+DL'].sum(), index = ['Total UL+DL'])
decile1



In [ ]:
decile2 = dur_agg[dur_agg['Decile_rank']==1]
decile2
numbers2 = df_user["Total UL+DL"]
decile2 = decile2.join(numbers2)
decile2.loc['Total'] = pd.Series(decile2['Total UL+DL'].sum(), index = ['Total UL+DL'])
decile2


In [ ]:
decile3 = dur_agg[dur_agg['Decile_rank']==2]
numbers3 = df_user["Total UL+DL"]
decile3 = decile3.join(numbers3)
decile3.loc['Total'] = pd.Series(decile3['Total UL+DL'].sum(), index = ['Total UL+DL'])
decile3


In [ ]:
decile4 = dur_agg[dur_agg['Decile_rank']==3]
numbers4 = df_user["Total UL+DL"]
decile4 = decile4.join(numbers4)
decile4.loc['Total'] = pd.Series(decile4['Total UL+DL'].sum(), index = ['Total UL+DL'])
decile4

In [ ]:
decile5 = dur_agg[dur_agg['Decile_rank']==4]
numbers5 = df_user["Total UL+DL"]
decile5 = decile5.join(numbers5)
decile5.loc['Total'] = pd.Series(decile5['Total UL+DL'].sum(), index = ['Total UL+DL'])
decile5


##AVG RTT

In [ ]:
sum_column_10 = df_user["Avg RTT DL (ms)"] + df_user["Avg RTT UL (ms)"]
df_user["Total Avg RTT"] = sum_column_10
rtt_df=df_user.groupby('MSISDN/Number')['Total Avg RTT'].sum().reset_index(name='Total_Avg_RTT')
rtt_df


#top 10
r1=rtt_df.nlargest(10,'Total_Avg_RTT')
r1.to_csv("r10.csv")



In [ ]:
#bottom 10
r2=rtt_df.nsmallest(10,'Total_Avg_RTT')
r2.to_csv("r10small.csv")

In [ ]:
#PLOT
df_user1=df_user.sample(frac =.25)
fig_dims = (16, 6)
fig, ax = plt.subplots(figsize=fig_dims)
sns.barplot(x='Handset Type', y='Total Avg RTT',ax=ax,data=df_user1)


##TCP

In [ ]:
sum_column11 = df_user["TCP DL Retrans. Vol (Bytes)"] + df_user["TCP UL Retrans. Vol (Bytes)"]
df_user["Total TCP Retrans"] = sum_column11
tcp_df=df_user.groupby('MSISDN/Number')['Total TCP Retrans'].sum().reset_index(name='Total_TCP_Retrans')
tcp_df


#top 10
t1=tcp_df.nlargest(10,'Total_TCP_Retrans')
t1.to_csv("tcp10.csv")

In [ ]:
t2=tcp_df.nsmallest(10,'Total_TCP_Retrans')
t2.to_csv("tcp10small.csv")

In [ ]:
tcp_df.mode()['Total_TCP_Retrans'][0]


In [ ]:
#PLOT
fig_dims = (16, 6)
fig, ax = plt.subplots(figsize=fig_dims)
sns.barplot(x='Handset Type', y='Total TCP Retrans',ax=ax,data=df_user)


##Handset Type

In [ ]:
handset_df=df_user.groupby('MSISDN/Number')['Handset Type'].count().reset_index(name='Handset count')
handset_df


data_h = [df_user["Handset Type"],handset_df["Handset count"]]
headers = ["Handset Type","Handset count" ]     
hdf11 = pd.concat(data_h, axis=1, keys=headers)
hdf11
hdf11.sort_values(['Handset Type', 'Handset count'], ascending=False).groupby('Handset count')


In [ ]:
hdf11.nlargest(25,'Handset count')



##Throughput

In [ ]:
sum_column12 = df_user["Avg Bearer TP DL (kbps)"] + df_user["Avg Bearer TP UL (kbps)"]
df_user["Total Avg TP"] = sum_column12
tp_df=df_user.groupby('MSISDN/Number')['Total Avg TP'].sum().reset_index(name='Total_Avg_TP_(kbps)')
tp_df

#top 10
tp1=tp_df.nlargest(10,'Total_Avg_TP_(kbps)')
tp1.to_csv("tp10.csv")
tp2=tp_df.nsmallest(10,'Total_Avg_TP_(kbps)')
tp2.to_csv("tpsmall.csv")


In [ ]:
tp_data= [tp_df["Total_Avg_TP_(kbps)"],hdf11["Handset Type"]]
headers122 = ["Total Avg Throughput","Handset Type"]  
thr_df=pd.concat(tp_data,axis=1,keys=headers122)
thr_df.to_csv("thr_h.csv")
# sns.barplot(y="Total Avg Throughput",x="Handset Type",data=thr_df)


In [ ]:
tp_data= [rtt_df["Total_Avg_RTT"],hdf11["Handset Type"]]
headers122 = ["Total Avg RTT","Handset Type"]  
thr_df1=pd.concat(tp_data,axis=1,keys=headers122)
thr_df1.to_csv("rttt.csv")


In [ ]:
tp_data= [tcp_df["Total_TCP_Retrans"],hdf11["Handset Type"]]
headers122 = ["Total TCP Retrans","Handset Type"]  
thr_df1=pd.concat(tp_data,axis=1,keys=headers122)
thr_df1.to_csv("tcp.csv")

##CORRELATION ANALYSIS

In [ ]:
#Correlation Analysis
data = [sm_df["Total_Social_Media_Volume"], gg_df["Total_Google_Volume"],em_df["Total Email volume (Bytes)"],yt_df["Total Youtube Volume (Bytes)"],nf_df["Total Netflix volume (Bytes)"],
        gm_df["Total Gaming Volume (Bytes)"],other_df["Total Other Volume(Bytes)"]]
headers = ["Total_Social_Media_Volume", "Total_Google_Volume",
           "Total Email volume (Bytes)","Total Youtube Volume (Bytes)",
           "Total Netflix volume (Bytes)","Total Gaming Volume (Bytes)",
          "Total Other Volume(Bytes)" ]     
matrix_df = pd.concat(data, axis=1, keys=headers)

# matrix_df.loc['Total'] = pd.Series(matrix_df['Total Email volume (Bytes)'].sum(), index = ['Total Email volume (Bytes)'])
# matrix_df
matrix1=matrix_df.sum(axis = 0, skipna = True) 
matrix1.to_csv("mat.csv",header=['sum'])
matrix2=matrix1.to_frame()
matrix2


type(matrix1)

corrMatrix = matrix_df.corr()
sns.heatmap(corrMatrix, annot=True)
plt.show()

In [ ]:
matrix1
matrix1.sort_values(inplace=True, ascending=False)
matrix3=matrix1.to_frame()
matrix3.head(3)


In [ ]:

matrix2.plot.bar(rot=0)


##PCA 

In [ ]:
from sklearn import decomposition

pca = decomposition.PCA(n_components=2)
pc = pca.fit_transform(matrix_df)
pc_df = pd.DataFrame(data = pc , 
        columns = ['PC1', 'PC2'])
pc_df.head()


In [ ]:
dd = pd.DataFrame({'var':pca.explained_variance_ratio_,
             'PC':['PC1','PC2']})
sns.barplot(x='PC',y="var", 
           data=dd, color="c");

In [ ]:
sns.lmplot( x="PC1", y="PC2",
  data=pc_df, 
  fit_reg=False, 
  legend=True,
  scatter_kws={"s": 80})


##K-means engagement(Experience Analytics)


In [ ]:
data1 = [rtt_df["MSISDN/Number"],tp_df["Total_Avg_TP_(kbps)"],tcp_df["Total_TCP_Retrans"],rtt_df["Total_Avg_RTT"]]
headers = [ "MSISDN/Number","Total_Avg_TP_(kbps)", "Total_TCP_Retrans",
           "Total_Avg_RTT",
           ]     
experience_df = pd.concat(data1, axis=1, keys=headers)
experience_df.fillna(experience_df.mean(), inplace=True)
experience_df

norm_experience=(experience_df-experience_df.min())/(experience_df.max()-experience_df.min())


kmeans1 = KMeans(n_clusters=3)

y = kmeans1.fit_predict(norm_experience[['Total_Avg_TP_(kbps)', 'Total_TCP_Retrans', 'Total_Avg_RTT',]])

norm_experience['Cluster'] = y


# print(experience_df)

df00 = norm_experience[norm_experience.Cluster==0]
df00
df11 =  norm_experience[norm_experience.Cluster==1]
df11
df22 =  norm_experience[norm_experience.Cluster==2]
df22



In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
scatter = ax.scatter(norm_experience['Total_Avg_TP_(kbps)'],norm_experience['Total_TCP_Retrans'],
                     c=kmeans.labels_,s=50)
ax.set_title('K-Means Clustering experience clusters')
ax.set_xlabel('Total_TCP_Retrans')
ax.set_ylabel('Total_Avg_TP_(kbps)')
plt.colorbar(scatter)

In [ ]:
norm_experience['Cluster'].value_counts()

In [ ]:
alldistances1 = kmeans1.fit_transform(norm_experience)
# alldistances
ddf2=pd.DataFrame(alldistances1)
ddf2.columns = ['0','1','2']
ddf2

##experience score

In [ ]:
#experience scores

experience_data= [norm_experience["MSISDN/Number"],ddf2["1"]]
headers11 = ["MSISDN/Number","experience_score"]  
experience_score=pd.concat(experience_data,axis=1,keys=headers11)
experience_score


##satisfaction score

In [ ]:
satisfaction_data= [xdr["MSISDN/Number"],experience_score["experience_score"],ddf["1"]]
satisfaction_data
headers22=['MSISDN/Number',"experience_score","engagement_score"]
satisfaction_score=pd.concat(satisfaction_data,axis=1,keys=headers22)
satisfaction_score

satisfaction_score['satisfaction_score'] = satisfaction_score[['experience_score', 'engagement_score']].mean(axis=1)
satisfaction_score.to_csv("satis.csv")

In [ ]:
s1=satisfaction_score.nlargest(10,'satisfaction_score')
s1.to_csv("s.csv")

## Regression 



In [ ]:
X = satisfaction_score['experience_score']  
Y = satisfaction_score['engagement_score']

X = X.values.reshape(-1, 1)
Y = Y.values.reshape(-1, 1)
 # with sklearn
regr = linear_model.LinearRegression()
regr.fit(X, Y)

print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)



# New_experience_score = 2.004168
# New_engagement_score = 1.638166	
# print ('Predicted Stock Index Price: \n', regr.predict([[New_experience_score ,New_engagement_score]]))




X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 
 
print_model = model.summary()
print(print_model)

##k=means satisfactory analysis


In [ ]:

kmeans1 = KMeans(n_clusters=2)

y = kmeans1.fit_predict(satisfaction_score[['experience_score', 'engagement_score']])

satisfaction_score['Cluster'] = y
satisfaction_score

In [ ]:
##average per cluster
df001 = satisfaction_score[satisfaction_score.Cluster==0]
df001

print("average experience score",df001["experience_score"].mean())
print("average satisfaction score",df001["satisfaction_score"].mean())


In [ ]:
df111 = satisfaction_score[satisfaction_score.Cluster==0]
df111["satisfaction_score"].mean() 


In [ ]:
df111 = satisfaction_score[satisfaction_score.Cluster==0]
df111["experience_score"].mean() 


In [ ]:
df112 = satisfaction_score[satisfaction_score.Cluster==1]
df112["satisfaction_score"].mean()

In [ ]:
df112 = satisfaction_score[satisfaction_score.Cluster==1]
df112["experience_score"].mean()